# Set Up

In [1]:
# Install module needed for the project
!pip install geoalchemy2
!pip install geopandas
!pip install psycopg2-binary

^C
^C
^C


In [5]:
# Import statements needed for the project
import json
import pathlib
import urllib.parse
import os

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db

from sqlalchemy.orm import declarative_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, String, Integer, Date, Float, MetaData, Table, create_engine
import psycopg2

# Part 1: Data Preprocessing

## 1.1) NYC Open Data

In [6]:
## Store URL and application token
url_complaint = "https://data.cityofnewyork.us/resource/erm2-nwe9.geojson"
url_tree = "https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson"
app_token = "YgXUUmVq41Z9433qxe5qpLOMG"

# Define date range
start_date = "2022-10-01"
end_date = "2023-09-30"

## Define parameter
params_complaint = {"$$app_token": app_token,
                    "$select": "incident_zip,created_date,location,complaint_type",
                    "$where": f"created_date between '{start_date}T00:00:00' and '{end_date}T23:59:59'",
                    "$limit": 1000}

params_tree = {"$$app_token": app_token,
               "$select": "zipcode,created_at, the_geom, spc_common,health,status",
               "$limit": 1000}

## Import NYC Open Data
raw_data_complaint = requests.get(url_complaint, params=params_complaint).json()
raw_data_tree = requests.get(url_tree, params=params_tree).json()

# Convert JSON to DataFrame
gdf_complaint = gpd.GeoDataFrame.from_features(raw_data_complaint['features'])
gdf_tree = gpd.GeoDataFrame.from_features(raw_data_tree['features'])

In [7]:
# Rename columns to have consistent naming
gdf_complaint.rename(columns={'incident_zip': 'zipcode'}, inplace=True)

# Rename columns to have consistent naming
gdf_complaint.rename(columns={'created_date': 'created_at'}, inplace=True)


# Convert data types if necessary (for example, ensuring zip_code is a string)
gdf_complaint['zipcode'] = gdf_complaint['zipcode'].astype(str)
gdf_tree['zipcode'] = gdf_tree['zipcode'].astype(str)


# Convert 'created_at' to datetime data type in complaint dataframe
gdf_complaint['created_at'] = pd.to_datetime(gdf_complaint['created_at'])


In [8]:
gdf_complaint

,geometry,zipcode,created_at,complaint_type
0,POINT (-73.95918 40.65567),11226,2023-09-30 23:59:58,Noise - Street/Sidewalk
1,POINT (-73.78752 40.76676),11361,2023-09-30 23:59:38,Noise - Residential
2,POINT (-73.98487 40.71950),10002,2023-09-30 23:59:35,Noise - Commercial
3,POINT (-73.79729 40.68750),11435,2023-09-30 23:59:34,Noise - Residential
4,POINT (-73.95795 40.65220),11226,2023-09-30 23:59:28,Noise - Residential
...,...,...,...,...
995,POINT (-73.89735 40.75889),11370,2023-09-30 22:35:17,Noise - Residential
996,POINT (-73.98164 40.76330),10019,2023-09-30 22:35:17,Noise - Vehicle
997,POINT (-73.95548 40.80428),10026,2023-09-30 22:35:07,Noise - Street/Sidewalk
998,POINT (-73.91600 40.81874),10455,2023-09-30 22:35:04,Illegal Parking


In [9]:
gdf_tree

,geometry,health,zipcode,spc_common,status,created_at
0,POINT (-73.84422 40.72309),Fair,11375,red maple,Alive,08/27/2015
1,POINT (-73.81868 40.79411),Fair,11357,pin oak,Alive,09/03/2015
2,POINT (-73.93661 40.71758),Good,11211,honeylocust,Alive,09/05/2015
3,POINT (-73.93446 40.71354),Good,11211,honeylocust,Alive,09/05/2015
4,POINT (-73.97598 40.66678),Good,11215,American linden,Alive,08/30/2015
...,...,...,...,...,...,...
995,POINT (-74.12944 40.56929),None,10306,None,Dead,08/31/2015
996,POINT (-74.00015 40.68505),Poor,11231,Norway maple,Alive,08/26/2015
997,POINT (-73.95752 40.62479),Good,11230,Norway maple,Alive,09/01/2015
998,POINT (-74.09660 40.59259),Good,10304,Norway maple,Alive,08/26/2015


## 1.2) Geometric boundary data 

In [10]:
### Define the path to the SHP file (and related files)
# Path for Irene  'data', 'nyc_zipcodes.shp'
# Path for Sahaphon r'C:\Users\USER\Documents\GitHub\Tools-for-Analytic-Project\data\nyc_zipcodes.shp'
Geom_file_path = os.path.join(os.getcwd(),'data', 'nyc_zipcodes.shp')

### Load the shapefile using geopandas and normalize 
gdf_zipcode = gpd.read_file(Geom_file_path).to_crs(epsg=4326) 
gdf_zipcode

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,10310,0,Staten Island,25003.0,5.346328e+07,NY,Richmond,36,085,http://www.usps.com/,0.0,0.0,"POLYGON ((-74.12065 40.64104, -74.12057 40.641..."
259,11693,0,Far Rockaway,11052.0,3.497516e+06,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.84076 40.62536, -73.84306 40.627..."
260,11249,0,Brooklyn,28481.0,1.777221e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.95805 40.72442, -73.95772 40.724..."
261,10162,1,New York,0.0,2.103489e+04,NY,New York,36,061,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.95133 40.76931, -73.95165 40.769..."


In [11]:
### Define the path to the SHP file (and related files)
# Path for Irene 'data', 'nyc_zipcodes.shp'
# Path for Sahaphon r'C:\Users\USER\Documents\GitHub\Tools-for-Analytic-Project\data\nyc_zipcodes.shp'
Geom_file_path = os.path.join(os.getcwd(),'data', 'nyc_zipcodes.shp')

### Load the shapefile using geopandas
gdf_zipcode = gpd.read_file(Geom_file_path)
gdf_zipcode

# Specify the columns you want to keep
columns_to_keep = ['ZIPCODE', 'geometry']
# Use the indexing operator to select the desired columns
gdf_zipcode = gdf_zipcode[columns_to_keep]
# Rename columns to have consistent naming
gdf_zipcode.rename(columns={'ZIPCODE': 'zipcode'}, inplace=True)

# Rename columns to have consistent naming
gdf_zipcode.rename(columns={'geometry': 'boundary'}, inplace=True)

# Convert data types if necessary
gdf_zipcode['zipcode'] = gdf_zipcode['zipcode'].astype(str)
gdf_zipcode

,zipcode,boundary
0,11436,"POLYGON ((1038098.252 188138.380, 1038141.936 ..."
1,11213,"POLYGON ((1001613.713 186926.440, 1002314.243 ..."
2,11212,"POLYGON ((1011174.276 183696.338, 1011373.584 ..."
3,11225,"POLYGON ((995908.365 183617.613, 996522.848 18..."
4,11218,"POLYGON ((991997.113 176307.496, 992042.798 17..."
...,...,...
258,10310,"POLYGON ((950767.507 172848.969, 950787.510 17..."
259,11693,"POLYGON ((1028453.995 167153.410, 1027813.010 ..."
260,11249,"POLYGON ((995877.318 203206.075, 995968.511 20..."
261,10162,"POLYGON ((997731.761 219560.922, 997641.948 21..."


## 1.3) Rent data 

In [12]:
### Define the path to the CSV file
# Path for Irene 'data', 'zillow_rent_data.csv'
# Path for Sahaphon r'C:\Users\USER\Documents\GitHub\Tools-for-Analytic-Project\data\zillow_rent_data.csv'
rent_file_path = os.path.join(os.getcwd(),'data', 'zillow_rent_data.csv')

### Convert CSV to DataFrame
df = pd.read_csv(rent_file_path)

# Define the columns you want to delete (replace with your actual column names)
columns_to_delete = ['SizeRank', 'RegionID',
                    'RegionType', 'StateName', 'State',
                    'City', 'Metro', 'CountyName',]

# Rename columns to have consistent naming
df.rename(columns={'RegionName': 'zipcode'}, inplace=True)

# Use the drop method to delete the specified columns
df = df.drop(columns=columns_to_delete)
df

,zipcode,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,77494,1606.206406,1612.779844,1622.201575,1630.392427,1632.411500,1636.206864,1644.894632,1643.390762,1636.971443,...,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,77449,1257.814660,1255.268025,1262.170452,1274.955754,1285.526052,1295.665673,1296.650395,1300.868081,1301.898486,...,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
2,77084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1701.217520,1706.900064,1706.067787,1723.722320,1735.484670,1752.132904,1756.990323,1754.429516,1757.602011,1755.031490
3,79936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1419.480272,1458.063897,1471.726681,1466.734658,1456.175660,1462.478506,1466.267391,1490.237063,1488.180414,1494.366097
4,11385,NaN,2087.527084,NaN,2149.924252,2166.263698,2148.992886,2190.098591,2264.966715,2297.900917,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6717,89158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3281.330738,3509.210744,3407.499896,3438.041504,3436.371804,3524.703410,3426.708975,3412.249969,3310.302151,3448.166667
6718,32461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2583.675563,2590.977335,2639.938102,2702.500000
6719,2876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2250.000000
6720,76005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2148.224601,2169.143026,2179.393248,2226.624684,2369.532530,2374.713926,2414.638428,2389.749852,2383.185013,2313.944444


In [13]:
# Convert zipcode to string
df['zipcode'] = df['zipcode'].astype(str)

# Reshape DataFrame from wide to long format
df_zillow = df.melt(id_vars='zipcode', var_name='date', value_name='average_rent')

# Convert 'date' to a datetime type
df_zillow['date'] = pd.to_datetime(df_zillow['date'], format='%Y-%m-%d')

print(df_zillow.head())


  zipcode       date  average_rent
0   77494 2015-01-31   1606.206406
1   77449 2015-01-31   1257.814660
2   77084 2015-01-31           NaN
3   79936 2015-01-31           NaN
4   11385 2015-01-31           NaN


In [14]:
## Part 2: Storing Data

In [15]:
## Create a new database in PostgreSQL
#!createdb group48project

In [16]:
## Turn on the PostGIS extension
#!psql --dbname group48project -c 'CREATE EXTENSION postgis;'

In [39]:
## Create Schema File (schema.sql):
ZIPCODE_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_zip_codes (
    id SERIAL PRIMARY KEY,
    boundary GEOMETRY(Polygon),
    zipcode VARCHAR(10)
);
"""



NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS complaints_311 (
    complaint_id SERIAL PRIMARY KEY,
    geometry GEOMETRY(Point),
    zipcode VARCHAR(10),
    created_at TIMESTAMP
);
"""

NYC_TREE_SCHEMA = """
CREATE TABLE IF NOT EXISTS trees (
    zipcode VARCHAR(10),
    created_at DATE,
    geometry GEOMETRY(Point),
    spc_common VARCHAR(255),
    health VARCHAR(50),
    status VARCHAR(50)
);
"""

ZILLOW_SCHEMA = """
CREATE TABLE IF NOT EXISTS historical_rents (
    id SERIAL PRIMARY KEY,
    zipcode VARCHAR(10),
    date DATE,
    average_rent NUMERIC
);  
"""

In [40]:
# create that required schema.sql file
with open('schema.sql', 'w') as file:
    file.write(ZIPCODE_SCHEMA)
    file.write('\n')
    file.write(NYC_311_SCHEMA)
    file.write('\n')
    file.write(NYC_TREE_SCHEMA)
    file.write('\n')
    file.write(ZILLOW_SCHEMA)

In [41]:
from sqlalchemy import text

In [35]:
from sqlalchemy import create_engine, text
from geoalchemy2 import Geometry
import pandas as pd
from shapely import wkb

# Database credentials
dbname = 'group48project'
user = 'postgres'
host = 'localhost'
port = '5432'
password = 'postgres'

# Create an SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Execute the schema creation SQL
with engine.connect() as connection:
    connection.execute(text(NYC_311_SCHEMA))

# Filter Out Invalid Geometries
gdf_complaint = gdf_complaint[gdf_complaint['geometry'].notnull()]

# Use the known SRID since we have normalized the data
srid = 4326

# Convert the 'geometry' column to WKB format
gdf_complaint['geometry_wkb'] = gdf_complaint['geometry'].apply(lambda geom: wkb.dumps(geom).hex())

# Define the table name
table_name = 'complaints_311'

# Directly insert the GeoDataFrame into the database
insert_query = f"INSERT INTO {table_name} (geometry, zipcode, created_at) VALUES (ST_GeomFromWKB(:geometry_wkb, {srid}), :zipcode, :created_at)"
with engine.connect() as connection:
    for index, row in gdf_complaint.iterrows():
        params = {
            'geometry_wkb': bytes.fromhex(row['geometry_wkb']),
            'zipcode': row['zipcode'],
            'created_at': row['created_at']
        }
        connection.execute(text(insert_query), params)

# Query to view the contents of the table
with engine.connect() as connection:
    query = text(f"SELECT ST_AsText(ST_GeomFromWKB(geometry::geometry)) as geometry, zipcode, created_at FROM {table_name};")
    result_df = pd.read_sql_query(query, connection)

# Display the first few rows of the DataFrame
print(result_df.head())

# Close the engine
engine.dispose()

                                       geometry zipcode          created_at
0  POINT(-73.95917686020623 40.655672001198894)   11226 2023-09-30 23:59:58
1   POINT(-73.78751847563191 40.76675595839554)   11361 2023-09-30 23:59:38
2   POINT(-73.98486650733275 40.71949965458691)   10002 2023-09-30 23:59:35
3   POINT(-73.7972903094197 40.687499303408536)   11435 2023-09-30 23:59:34
4    POINT(-73.9579464603267 40.65220215349917)   11226 2023-09-30 23:59:28


In [42]:
# Execute the schema creation SQL
with engine.connect() as connection:
    connection.execute(text(ZIPCODE_SCHEMA))

# Filter Out Invalid Geometries
gdf_zipcode = gdf_zipcode[gdf_zipcode['boundary'].notnull()]

# Convert the 'geometry' column to WKB format
gdf_zipcode['boundary_wkb'] = gdf_zipcode['boundary'].apply(lambda geom: wkb.dumps(geom).hex())

# Define the table name
table_name = 'nyc_zip_codes'

# Directly insert the GeoDataFrame into the database
insert_query = f"INSERT INTO {table_name} (zipcode, boundary) VALUES (:zipcode, ST_GeomFromWKB(:boundary_wkb, {srid}))"
with engine.connect() as connection:
    for index, row in gdf_zipcode.iterrows():
        params = {
            'zipcode': row['zipcode'],
            'boundary_wkb': bytes.fromhex(row['boundary_wkb']),
        }
        connection.execute(text(insert_query), params)
        
        
# Query to view the contents of the table
with engine.connect() as connection:
    query = text(f"SELECT zipcode, ST_AsText(ST_GeomFromWKB(boundary::geometry)) as boundary FROM {table_name};")
    result_df = pd.read_sql_query(query, connection)

# Display the first few rows of the DataFrame
print(result_df.head())

# Close the engine
engine.dispose()

  zipcode                                           boundary
0   11436  POLYGON((1038098.2518714815 188138.38000671566...
1   11213  POLYGON((1001613.7129640579 186926.4395172149,...
2   10009  POLYGON((991606.4032543153 205494.14647620916,...
3   11212  POLYGON((1011174.275535807 183696.33770971,101...
4   11225  POLYGON((995908.3654508889 183617.61280155182,...


In [37]:
# Storing tree data
# Convert 'created_at' column to DATE format
from sqlalchemy import create_engine, text
from geoalchemy2 import Geometry
import pandas as pd
from shapely import wkb

# Database credentials
dbname = 'group48project'
user = 'postgres'
host = 'localhost'
port = '5432'
password = 'postgres'

# Create an SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Execute the schema creation SQL
with engine.connect() as connection:
    connection.execute(text(NYC_TREE_SCHEMA))
    
    
    
gdf_tree['created_at'] = pd.to_datetime(gdf_tree['created_at']).dt.date

# Filter Out Invalid Geometries
gdf_tree = gdf_tree[gdf_tree['geometry'].notnull()]

# Use the known SRID since we have normalized the data
srid = 4326
    
# Convert the 'geometry' column to WKB format
gdf_tree['geometry_wkb'] = gdf_tree['geometry'].apply(lambda geom: wkb.dumps(geom).hex())

# Define the table name for trees
table_name_trees = 'trees'
    
# Insert data into the 'trees' table
insert_query_trees = f"INSERT INTO {table_name_trees} (zipcode, created_at, geometry, spc_common, health, status) VALUES (:zipcode,:created_at, ST_GeomFromWKB(:geometry_wkb, {srid}), :spc_common, :health, :status)"
    
with engine.connect() as connection:
    for index, row in gdf_tree.iterrows():
        params = {
            'zipcode': row['zipcode'],
            'created_at': row['created_at'],
            'geometry_wkb': bytes.fromhex(row['geometry_wkb']),
            'spc_common': row['spc_common'],
            'health': row['health'],
            'status': row['status']
        }
        connection.execute(text(insert_query_trees), params)

# Query to view the contents of the 'trees' table
with engine.connect() as connection:
    query_trees = text(f"SELECT zipcode, created_at, ST_AsText(ST_GeomFromWKB(geometry::geometry)) as geometry, spc_common, health, status FROM {table_name_trees};")
    result_df_trees = pd.read_sql_query(query_trees, connection)

# Display the first few rows of the DataFrame
print(result_df_trees.head())

  zipcode  created_at                                      geometry  \
0   11375  2015-08-27  POINT(-73.84421521958048 40.723091773924274)   
1   11357  2015-09-03   POINT(-73.81867945834878 40.79411066708779)   
2   11211  2015-09-05  POINT(-73.93660770459083 40.717580740099116)   
3   11211  2015-09-05  POINT(-73.93445615919741 40.713537494833226)   
4   11215  2015-08-30   POINT(-73.97597938483258 40.66677775537875)   

        spc_common health status  
0        red maple   Fair  Alive  
1          pin oak   Fair  Alive  
2      honeylocust   Good  Alive  
3      honeylocust   Good  Alive  
4  American linden   Good  Alive  


In [38]:
# STORE ZILLOW DATA
import pandas as pd
from sqlalchemy import create_engine

# Database credentials
dbname = 'group48project'
user = 'postgres'
host = 'localhost'
port = '5432'
password = 'postgres'

# Create an SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Insert data into the database
df_zillow.to_sql('historical_rents', con=engine, if_exists='append', index=False)

# Read 'historical_rents' table into a DataFrame
df_from_sql = pd.read_sql_table('historical_rents', con=engine)

# Display the first few rows of the DataFrame
print(df_from_sql.head())


  zipcode       date  average_rent
0   77494 2015-01-31   1606.206406
1   77449 2015-01-31   1257.814660
2   77084 2015-01-31           NaN
3   79936 2015-01-31           NaN
4   11385 2015-01-31           NaN


In [44]:
from sqlalchemy import create_engine, text
import pandas as pd

# Database credentials
dbname = 'group48project'
user = 'postgres'
host = 'localhost'
port = '5432'
password = 'postgres'

# Create an SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# SQL query to find top 10 zip codes with the most trees
query_top_zipcodes = """
    SELECT
        zipcode,
        COUNT(*) AS total_trees
    FROM
        trees
    GROUP BY
        zipcode
    ORDER BY
        total_trees DESC
    LIMIT 10;
"""

# Execute the query
with engine.connect() as connection:
    result_df_top_zipcodes = pd.read_sql_query(text(query_top_zipcodes), connection)

# Display the result
print(result_df_top_zipcodes)


  zipcode  total_trees
0   11375          152
1   11215          140
2   11105          140
3   10023          132
4   10457          124
5   11205          120
6   11226          116
7   10306          112
8   11230           96
9   10458           88


In [47]:
from sqlalchemy import create_engine, text
import pandas as pd

# Database credentials
dbname = 'group48project'
user = 'postgres'
host = 'localhost'
port = '5432'
password = 'postgres'

# Create an SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# SQL query to find top 10 zip codes with the most trees and average rent for August 2023
query_top_zipcodes_with_rent = """
    SELECT
        t.zipcode,
        COUNT(*) AS total_trees,
        to_char(AVG(hr.average_rent), 'FM999,999,999.99') AS average_rent
    FROM
        trees t
    JOIN
        historical_rents hr ON t.zipcode = hr.zipcode
    WHERE
        EXTRACT(MONTH FROM hr.date) = 8 AND EXTRACT(YEAR FROM hr.date) = 2023
    GROUP BY
        t.zipcode
    ORDER BY
        total_trees DESC
    LIMIT 10;
"""

# Execute the query
with engine.connect() as connection:
    result_df_top_zipcodes_with_rent = pd.read_sql_query(text(query_top_zipcodes_with_rent), connection)

# Display the result
print(result_df_top_zipcodes_with_rent)


  zipcode  total_trees average_rent
0   11375          152      2,743.4
1   11215          140     3,575.65
2   11105          140     2,852.73
3   10023          132     4,370.07
4   10457          124     2,183.97
5   11205          120     3,497.47
6   11226          116     2,785.32
7   10306          112     2,331.54
8   11230           96     2,657.04
9   10024           88     3,797.94


In [48]:
from sqlalchemy import text

# SQL query to find the 5 zip codes with the lowest and highest average rent in January 2023
query_low_high_avg_rent = """
    WITH RentTreeComplaintData AS (
        SELECT
            hr.zipcode,
            to_char(AVG(hr.average_rent), 'FM999,999,999.99') AS avg_rent,
            COUNT(DISTINCT t.geometry) AS tree_count,
            COUNT(DISTINCT c.complaint_id) AS complaint_count
        FROM
            historical_rents hr
        LEFT JOIN
            trees t ON hr.zipcode = t.zipcode
        LEFT JOIN
            complaints_311 c ON hr.zipcode = c.zipcode
        WHERE
            EXTRACT(MONTH FROM hr.date) = 1 AND EXTRACT(YEAR FROM hr.date) = 2023
        GROUP BY
            hr.zipcode
    )
    SELECT
        r.zipcode,
        r.avg_rent,
        COALESCE(rtcd.tree_count, 0) AS tree_count,
        COALESCE(rtcd.complaint_count, 0) AS complaint_count
    FROM
        (SELECT * FROM RentTreeComplaintData ORDER BY avg_rent ASC LIMIT 5) r
    FULL JOIN
        (SELECT * FROM RentTreeComplaintData ORDER BY avg_rent DESC LIMIT 5) rtcd
    ON
        r.zipcode = rtcd.zipcode
    ORDER BY
        r.avg_rent ASC;
"""

# Execute the query
with engine.connect() as connection:
    result_df_low_high_avg_rent = pd.read_sql_query(text(query_low_high_avg_rent), connection)

# Display the result
print(result_df_low_high_avg_rent)


  zipcode  avg_rent  tree_count  complaint_count
0   71270  1,000.82           0                0
1   53208  1,001.47           0                0
2   61265  1,003.08           0                0
3   35020  1,003.48           0                0
4   77901  1,003.71           0                0
5    None      None           0                0
6    None      None          19               68
7    None      None           0                0
8    None      None           0               34
9    None      None           0                0


In [49]:
from sqlalchemy import text
from shapely.geometry import Point
from geoalchemy2 import WKTElement

# Given coordinates
latitude = 40.80737875669467
longitude = -73.96253174434912
radius_miles = 0.5

# Convert the radius to degrees (assuming 1 degree is approximately 69 miles)
radius_degrees = radius_miles / 69.0

# Create a Shapely Point object for the given coordinates
point = Point(longitude, latitude)

# Create a WKT (Well-Known Text) representation of the Point
point_wkt = WKTElement(point.wkt, srid=4326)

# SQL query to find trees within ½ mile radius of the given point
query_trees_within_radius = f"""
    SELECT
        ID,
        spc_common AS species,
        health,
        status,
        ST_AsText(geometry) AS coordinate_location
    FROM
        trees
    WHERE
        ST_DWithin(geometry, ST_GeomFromEWKT(:point_wkt), :radius_degrees)
"""

# Execute the query
with engine.connect() as connection:
    params = {
        'point_wkt': point_wkt,
        'radius_degrees': radius_degrees
    }
    result_df_trees_within_radius = pd.read_sql_query(text(query_trees_within_radius), connection, params=params)

# Display the result
print(result_df_trees_within_radius)


ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'WKTElement'
[SQL: 
    SELECT
        ID,
        spc_common AS species,
        health,
        status,
        ST_AsText(geometry) AS coordinate_location
    FROM
        trees
    WHERE
        ST_DWithin(geometry, ST_GeomFromEWKT(%(point_wkt)s), %(radius_degrees)s)
]
[parameters: {'point_wkt': <WKTElement at 0x2655607b810; POINT (-73.96253174434912 40.80737875669467)>, 'radius_degrees': 0.007246376811594203}]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [50]:
from sqlalchemy import text
from shapely.geometry import Point
from geoalchemy2 import WKTElement

# Given coordinates
latitude = 40.80737875669467
longitude = -73.96253174434912
radius_miles = 0.5

# Convert the radius to degrees (assuming 1 degree is approximately 69 miles)
radius_degrees = radius_miles / 69.0

# Create a Shapely Point object for the given coordinates
point = Point(longitude, latitude)

# Create a WKT (Well-Known Text) representation of the Point
point_wkt = point.wkt

# SQL query to find trees within ½ mile radius of the given point
query_trees_within_radius = f"""
    SELECT
        ID,
        spc_common AS species,
        health,
        status,
        ST_AsText(geometry) AS coordinate_location
    FROM
        trees
    WHERE
        ST_DWithin(geometry, ST_GeomFromEWKT(:point_wkt), :radius_degrees)
"""

# Execute the query
with engine.connect() as connection:
    params = {
        'point_wkt': point_wkt,
        'radius_degrees': radius_degrees
    }
    result_df_trees_within_radius = pd.read_sql_query(text(query_trees_within_radius), connection, params=params)

# Display the result
print(result_df_trees_within_radius)


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "id" does not exist
LINE 3:         ID,
                ^

[SQL: 
    SELECT
        ID,
        spc_common AS species,
        health,
        status,
        ST_AsText(geometry) AS coordinate_location
    FROM
        trees
    WHERE
        ST_DWithin(geometry, ST_GeomFromEWKT(%(point_wkt)s), %(radius_degrees)s)
]
[parameters: {'point_wkt': 'POINT (-73.96253174434912 40.80737875669467)', 'radius_degrees': 0.007246376811594203}]
(Background on this error at: https://sqlalche.me/e/14/f405)